In [1]:
## Verzió megállapítása (URL-be kell illeszteni)
import requests
verzio_url = "https://vtr.valasztas.hu/ogy2022/data/config.json"
verzio_response = requests.get(verzio_url)
verzio = verzio_response.json().get("ver")
szavossz = verzio_response.json().get("szavossz")
print(verzio)
print(szavossz)

04022333
04161400


In [2]:
## Települések listájának lekérdezése
telepulesek_url = "https://vtr.valasztas.hu/ogy2022/data/" + verzio + "/ver/Telepulesek.json"
print(telepulesek_url)
telepulesek_response = requests.get(telepulesek_url)
telepulesek_response.json()

https://vtr.valasztas.hu/ogy2022/data/04022333/ver/Telepulesek.json


{'header': {'generated': '2022-04-02T23:30:00', 'vl_id': 333, 'nvv_id': 352},
 'list': [{'maz': '01',
   'taz': '001',
   'megnev': 'Budapest I. kerület',
   'evk_lst': ['01'],
   'szk_db': 20,
   'letszam': {'indulo': 20090,
    'honos': 18684,
    'atjel': 1265,
    'atjelInnen': 1311,
    'kuvi': 669,
    'osszesen': 19949}},
  {'maz': '01',
   'taz': '002',
   'megnev': 'Budapest II. kerület',
   'evk_lst': ['03', '04'],
   'szk_db': 74,
   'letszam': {'indulo': 64629,
    'honos': 60271,
    'atjel': 3081,
    'atjelInnen': 4052,
    'kuvi': 2280,
    'osszesen': 63352}},
  {'maz': '01',
   'taz': '003',
   'megnev': 'Budapest III. kerület',
   'evk_lst': ['04', '10'],
   'szk_db': 113,
   'letszam': {'indulo': 98416,
    'honos': 94166,
    'atjel': 3828,
    'atjelInnen': 3925,
    'kuvi': 1675,
    'osszesen': 97994}},
  {'maz': '01',
   'taz': '004',
   'megnev': 'Budapest IV. kerület',
   'evk_lst': ['11', '12'],
   'szk_db': 71,
   'letszam': {'indulo': 75425,
    'honos': 7

In [3]:
import pandas as pd
valasztas_df = pd.DataFrame()

## Megye azonosítók
megye_azonositok = {"01":"Budapest","02":"Bács-Kiskun","03":"Baranya","04":"Békés","05":"Borsod-Abaúj-Zemplén","06":"Csongrád-Csanád","07":"Fejér","08":"Győr-Moson-Sopron","09":"Hajdú-Bihar","10":"Heves","11":"Jász-Nagykun-Szolnok","12":"Komárom-Esztergom","13":"Nógrád","14":"Pest","15":"Somogy","16":"Szabolcs-Szatmár-Bereg","17":"Tolna","18":"Vas","19":"Veszprém","20":"Zala"}

## Települések és szavazókörök adatainak összegyűjtése
telepules_lista = telepulesek_response.json().get("list")
for telepules in telepules_lista:
    telepulesaz = telepules.get("maz") + "-" + telepules.get("taz")
    megyenev = megye_azonositok.get(telepules.get("maz"))
    telepulesnev = telepules.get("megnev")
        
    sor = {"telepulesaz":telepulesaz,"megyenev":megyenev,"telepulesnev":telepulesnev}
    valasztas_df = valasztas_df.append(sor, ignore_index=True)
valasztas_df

,telepulesaz,megyenev,telepulesnev
0,01-001,Budapest,Budapest I. kerület
1,01-002,Budapest,Budapest II. kerület
2,01-003,Budapest,Budapest III. kerület
3,01-004,Budapest,Budapest IV. kerület
4,01-005,Budapest,Budapest V. kerület
...,...,...,...
3172,20-254,Zala,Zalaújlak
3173,20-255,Zala,Zalavár
3174,20-256,Zala,Zalavég
3175,20-257,Zala,Zebecke


In [4]:
## Listák összegyűjtése
listak = {"950":"DK-JOBBIK-MOMENTUM-MSZP-LMP-PÁRBESZÉD", "955":"NORMÁLIS PÁRT", "951": "MKKP", "954":"MEMO", "942":"MI HAZÁNK","952":"FIDESZ-KDNP"}

In [5]:
## Listás eredmények összegyűjtése és hozzárendelés településekhez
telepules_eredmenyek = []
for index, row in valasztas_df.iterrows():
    df_megyeaz = row['telepulesaz'].split('-')[0]
    df_telepulesaz = row['telepulesaz']
    eredmenyek_url = "https://vtr.valasztas.hu/ogy2022/data/" + szavossz + "/szavossz/" + df_megyeaz + "/SzavkorJkv-"+ df_telepulesaz + ".json"
    eredmenyek_response = requests.get(eredmenyek_url)
    valasztopolgarok_szama = []
    szavazatok_szama = []

    szavazokorok = eredmenyek_response.json().get("list")
    partok_es_szavazatok = {}
    for listaid in listak.keys():
        partok_es_szavazatok[listaid] = []

    for x in szavazokorok:
        valasztopolgarok_szama.append(x.get("listas_jkv").get("vp_osszes"))
        szavazatok_szama.append(x.get("listas_jkv").get("szl_ervenyes"))
        tetelek = x.get("listas_jkv").get("tetelek")
        for part in tetelek:
            tl_id = str(part.get("tl_id"))
            if tl_id in partok_es_szavazatok.keys():
                jelenlegi_lista = partok_es_szavazatok[tl_id]
                jelenlegi_lista.append(part.get("szavazat"))
                partok_es_szavazatok[tl_id] = jelenlegi_lista

    teljes_osszeg = 0        
    for tl_id in partok_es_szavazatok.keys():
        szavazatok = partok_es_szavazatok[tl_id]
        osszeg = sum(szavazatok)
        teljes_osszeg = teljes_osszeg + osszeg
        partok_es_szavazatok[tl_id] = osszeg
    osszes_valasztopolgar = sum(valasztopolgarok_szama)
    osszes_szavazat = sum(szavazatok_szama)
    egyeb_szavazatok = osszes_szavazat - teljes_osszeg
    partok_es_szavazatok["Egyéb"] = egyeb_szavazatok
    partok_es_szavazatok["telepulesaz"] = row['telepulesaz']
    partok_es_szavazatok["valasztok_szama"] = osszes_valasztopolgar
    partok_es_szavazatok["szavazatok_szama"] = osszes_szavazat
    telepules_eredmenyek.append(partok_es_szavazatok)

In [6]:
## Választási eredmények dataframe-be helyezése
valasztas_eredmenyek = pd.DataFrame(telepules_eredmenyek)

In [7]:
## Lista azonosítók átnevezése pártnevekké
valasztas_eredmenyek.rename(listak, axis=1, inplace=True)
valasztas_eredmenyek

,DK-JOBBIK-MOMENTUM-MSZP-LMP-PÁRBESZÉD,NORMÁLIS PÁRT,MKKP,MEMO,MI HAZÁNK,FIDESZ-KDNP,Egyéb,telepulesaz,valasztok_szama,szavazatok_szama
0,6541,43,927,139,357,6406,25,01-001,18683,14438
1,25848,154,3048,433,1288,21013,124,01-002,64218,51908
2,36642,405,4008,993,2996,29902,120,01-003,96706,75066
3,27250,347,2680,874,2597,22021,74,01-004,75046,55843
4,7481,59,908,184,448,6699,40,01-005,20518,15819
...,...,...,...,...,...,...,...,...,...,...
3172,16,0,3,0,6,38,0,20-254,79,63
3173,93,2,13,0,27,305,1,20-255,699,441
3174,33,0,2,1,11,140,0,20-256,278,187
3175,5,1,0,0,2,23,0,20-257,44,31


In [8]:
vegleges_df = pd.merge(valasztas_df, valasztas_eredmenyek, how="inner", on="telepulesaz")
vegleges_df

,telepulesaz,megyenev,telepulesnev,DK-JOBBIK-MOMENTUM-MSZP-LMP-PÁRBESZÉD,NORMÁLIS PÁRT,MKKP,MEMO,MI HAZÁNK,FIDESZ-KDNP,Egyéb,valasztok_szama,szavazatok_szama
0,01-001,Budapest,Budapest I. kerület,6541,43,927,139,357,6406,25,18683,14438
1,01-002,Budapest,Budapest II. kerület,25848,154,3048,433,1288,21013,124,64218,51908
2,01-003,Budapest,Budapest III. kerület,36642,405,4008,993,2996,29902,120,96706,75066
3,01-004,Budapest,Budapest IV. kerület,27250,347,2680,874,2597,22021,74,75046,55843
4,01-005,Budapest,Budapest V. kerület,7481,59,908,184,448,6699,40,20518,15819
...,...,...,...,...,...,...,...,...,...,...,...,...
3172,20-254,Zala,Zalaújlak,16,0,3,0,6,38,0,79,63
3173,20-255,Zala,Zalavár,93,2,13,0,27,305,1,699,441
3174,20-256,Zala,Zalavég,33,0,2,1,11,140,0,278,187
3175,20-257,Zala,Zebecke,5,1,0,0,2,23,0,44,31


In [9]:
## Eredmények mentése CSV fájlba
vegleges_df.to_csv("valasztas_eredmenyek.csv", index=False)